# Keras Implementation

Moving to Keras for the implementation to quickly churn out models. The goal of this notebook is to achieve numbers similar/better to those achieved using TensorFlow.

In [1]:
# import the libraries

import numpy as np 
import matplotlib.pyplot as plt
import scipy
import scipy.io as sio
import scipy.ndimage as ndimage

%matplotlib inline
import math
import os
import pylab as P
import skimage
import h5py
import PIL.Image as Image
import functools
from sklearn.cross_validation import train_test_split as ttsplit
import cPickle as pickle
import tensorflow as tf

import keras # the point of this exercise

Using TensorFlow backend.


In [2]:
# helper function to read a pickle file
# TODO: move this to helpers file
def read_pickle_file(f_name):
    f = open(f_name, 'r')
    return pickle.load(f)


print os.getcwd()
data_dir = os.path.join(os.path.split(os.getcwd())[0], 'data')
print data_dir

def get_data_path(f_name):
    return os.path.join(data_dir, f_name)

/home/ag/projects/ml_nanodegree/clean_repo/capstone/src
/home/ag/projects/ml_nanodegree/clean_repo/capstone/data


In [3]:
file_list = [
    'train_single_data.pkl',
    'train_single_labels.pkl',
    'valid_single_data.pkl',
    'valid_single_labels.pkl',
    'test_single_data.pkl',
    'test_single_labels.pkl',
]


train_images, train_labels, valid_images, valid_labels, test_images, test_labels  = \
    [read_pickle_file(get_data_path(f_name)) for f_name in file_list]


In [34]:
print train_images.shape
print train_labels.shape
print test_images.shape
print test_labels.shape

(594368, 32, 32, 1)
(594368,)
(26032, 32, 32, 1)
(26032,)


In [4]:
label_set = set(train_labels)
assert(len(label_set) == 10)
num_labels = len(label_set)
labels = range(10)
assert(len(label_set.difference(set(labels))) == 0)

In [5]:
#define an accuracy measure for this model
def accuracy(predictions, labels):
    """
        How many places did we get right?
    """
    return (100.0 * np.sum(np.argmax(predictions, 1) == labels)) / labels.shape[0]

In [17]:
## time to define the model

from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, AveragePooling2D, Dropout, Flatten
from keras.optimizers import SGD

In [68]:
# create a sequential model
model = Sequential()

## let's add a conv and pooling layer
## let's follow the same model we used with TF

## convolve from 32x32x1 to 28x28x16
model.add(Convolution2D(16, 5, 5, border_mode='valid', input_shape=(32, 32, 1)))
# let's pool this - go to 14x14x16
model.add(MaxPooling2D(pool_size=(2,2), strides=None))
# pass through a filter
model.add(Activation('relu'))

## convolve and pool and filter again
## go to 32x10x10
model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(14, 14, 16)))
## go to 32x5x5
model.add(MaxPooling2D(pool_size=(2,2), strides=None))
model.add(Activation('relu'))

## and again
# go to 64x3x3
model.add(Convolution2D(64, 3, 3, border_mode='valid', input_shape=(5, 5, 32)))
# go to 64x1x1
model.add(MaxPooling2D(pool_size=(3,3), strides=None))
model.add(Activation('relu'))

## Flatten
model.add(Flatten())

## Dropout
model.add(Dropout(0.05))

## slap on the FCN at the end
## h1
model.add(Dense(output_dim=64, input_dim=64))
model.add(Activation("relu"))

# h2
model.add(Dense(output_dim=32, input_dim=128))
model.add(Activation("relu"))

## output
model.add(Dense(output_dim=len(labels), input_dim=32))
model.add(Activation("softmax"))


# specify the loss
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])



In [75]:
hist = model.fit(train_images, train_labels, batch_size=512, nb_epoch=10) # 60 epochs before this - > total 70

Epoch 1/10
594368/594368 [==============================] - 21s - loss: 0.0999 - acc: 0.9721    
Epoch 2/10
594368/594368 [==============================] - 21s - loss: 0.0994 - acc: 0.9725    
Epoch 3/10
594368/594368 [==============================] - 22s - loss: 0.0991 - acc: 0.9724    
Epoch 4/10
594368/594368 [==============================] - 21s - loss: 0.0986 - acc: 0.9725    
Epoch 5/10
594368/594368 [==============================] - 22s - loss: 0.0978 - acc: 0.9729    
Epoch 6/10
594368/594368 [==============================] - 22s - loss: 0.0978 - acc: 0.9729    
Epoch 7/10
594368/594368 [==============================] - 22s - loss: 0.0974 - acc: 0.9729    
Epoch 8/10
594368/594368 [==============================] - 21s - loss: 0.0969 - acc: 0.9731    
Epoch 9/10
594368/594368 [==============================] - 21s - loss: 0.0966 - acc: 0.9731    
Epoch 10/10
594368/594368 [==============================] - 21s - loss: 0.0965 - acc: 0.9732    


In [76]:
model.evaluate(test_images, test_labels)

25920/26032 [============================>.] - ETA: 0s

[0.21448607176436957, 0.94510602335586968]